In [1]:
!pip install faiss-cpu scikit-learn scipy

  Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.3.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 4.5 MB/s eta 0:00:0000:0100:01
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached threadpoolctl-3.3.0-py3-none-any.whl (17 kB)


In [5]:
import os
from zhipuai import ZhipuAI
print(os.environ["ZHIPUAI_API_KEY"])

client=ZhipuAI()

embedding_text="hello world"

response=client.embeddings.create(
    model="embedding-2",
    input=embedding_text
)

f50db9431477ef057558ce235c98a68d.l0Dxbc5k7fekpkch


In [6]:
response.usage.total_tokens

3

In [7]:
response.data[0].embedding

[-0.0686611533164978,
 -0.01344419177621603,
 -0.018471740186214447,
 0.022484784945845604,
 0.01903722994029522,
 -0.031042881309986115,
 -0.028715789318084717,
 0.047315776348114014,
 -0.023289412260055542,
 -0.0010944000678136945,
 0.006920468527823687,
 0.009118727408349514,
 0.0023793810978531837,
 0.038529764860868454,
 0.05440162494778633,
 -0.00675188098102808,
 0.01351057831197977,
 0.010793834924697876,
 -0.0024169080425053835,
 -0.04071984067559242,
 -0.026856200769543648,
 -0.009823048487305641,
 -0.00045497878454625607,
 -0.06797516345977783,
 0.022623222321271896,
 -0.037742991000413895,
 -0.01211057510226965,
 -0.03959466889500618,
 -0.005243110004812479,
 -0.05262289568781853,
 0.07098550349473953,
 0.012955510057508945,
 0.005307376384735107,
 -0.0066172112710773945,
 0.020235715433955193,
 0.03266850486397743,
 0.002741944743320346,
 0.019814468920230865,
 0.010425606742501259,
 -0.013362414203584194,
 -0.0029353101272135973,
 0.008259166963398457,
 0.0394467711448669

In [9]:
import numpy as np
import faiss
embedding_text="""
Multi-modal AI systems will likely become a ubiquitous presence in our everyday lives. A promising approach to making these systems more interactive is to embody them as agents within physical and virtual environments. At present, systems leverage existing foundation models as the basic building blocks for the creation of embodied agents. Embedding agents within such environments facilitates the ability of models to process and interpret visual and contextual data, which is critical for the creation of more sophisticated and context-aware AI systems. For example, a system that can perceive user actions, human behavior, environmental objects, audio expressions, and the collective sentiment of a scene can be used to inform and direct agent responses within the given environment. To accelerate research on agent-based multimodal intelligence, we define "Agent AI" as a class of interactive systems that can perceive visual stimuli, language inputs, and other environmentally-grounded data, and can produce meaningful embodied actions. In particular, we explore systems that aim to improve agents based on next-embodied action prediction by incorporating external knowledge, multi-sensory inputs, and human feedback. We argue that by developing agentic AI systems in grounded environments, one can also mitigate the hallucinations of large foundation models and their tendency to generate environmentally incorrect outputs. The emerging field of Agent AI subsumes the broader embodied and agentic aspects of multimodal interactions. Beyond agents acting and interacting in the physical world, we envision a future where people can easily create any virtual reality or simulated scene and interact with agents embodied within the virtual environment.
"""

chunk_size=50
chunks=[embedding_text[i:i+chunk_size] for i in range(0,len(embedding_text),chunk_size)]
chunks

['\nMulti-modal AI systems will likely become a ubiqu',
 'itous presence in our everyday lives. A promising ',
 'approach to making these systems more interactive ',
 'is to embody them as agents within physical and vi',
 'rtual environments. At present, systems leverage e',
 'xisting foundation models as the basic building bl',
 'ocks for the creation of embodied agents. Embeddin',
 'g agents within such environments facilitates the ',
 'ability of models to process and interpret visual ',
 'and contextual data, which is critical for the cre',
 'ation of more sophisticated and context-aware AI s',
 'ystems. For example, a system that can perceive us',
 'er actions, human behavior, environmental objects,',
 ' audio expressions, and the collective sentiment o',
 'f a scene can be used to inform and direct agent r',
 'esponses within the given environment. To accelera',
 'te research on agent-based multimodal intelligence',
 ', we define "Agent AI" as a class of interactive s',
 'ystems 

In [10]:
from sklearn.preprocessing import  normalize
import numpy as np
import faiss
embeddings=[]
for chunk in chunks:
    response=client.embeddings.create(
        model="embedding-2",
        input=chunk
    )
    embeddings.append(response.data[0].embedding)

normalized_embeddings=normalize(np.array(embeddings).astype('float32'))
d=1024
index=faiss.IndexFlatIP(d)
index.add(normalized_embeddings)

n_vectors=index.ntotal
n_vectors

36

In [12]:
from sklearn.preprocessing import  normalize
def match_text(input_text,index,chunks,k=2):
    k=min(k,len(chunks))
    response=client.embeddings.create(
        model="embedding-2",
        input=input_text
    )
    input_embedding=response.data[0].embedding
    input_embedding=normalize(np.array([input_embedding]).astype('float32'))

    distances,indices=index.search(input_embedding,k)

    for i,idx in enumerate(indices[0]):
        print(f"similarity:{distances[0][i]:4f}\n matching text:\n {chunks[idx]}")



input_text="perceive visual stimuli"

matched_texts=match_text(input_text=input_text,index=index,chunks=chunks,k=2)

similarity:0.720991
 matching text:
 ystems that can perceive visual stimuli, language 
similarity:0.602754
 matching text:
 ability of models to process and interpret visual 
